In [1]:
import sys
GCOLAB = 'google.colab' in sys.modules
if GCOLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from functools import reduce
import string
import nltk
import math
from tqdm import tqdm
!pip3 install gensim
import gensim
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import re
import collections
import copy
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!pip3 install autocorrect
!pip install catboost
from autocorrect import Speller
from catboost import CatBoostClassifier
GRID_SEARCH = False
GRID_SEARCH_BEST = False
SELECT_BEST_PREPOCESS = True

In [4]:
if GCOLAB:
  folder_path = './drive/MyDrive/21 School/tweets/data/'
else:
  folder_path = "./data/"
files = ['processedPositive.csv', 'processedNeutral.csv', 'processedNegative.csv']
tweets = dict()
for target in files:
  with open(folder_path + target, 'r') as file_:
    text = re.sub(r"\d+", "", file_.read().strip().lower())
  new_text = ''
  i = 0
  text = text.replace("\'", '').replace('\"', '')
  len_ = len(text)
  while i < len_:
    if text[i] == ','  and text[i + 1] not in ' ':
      new_text += '\n'
    else:
      new_text += text[i]
    i += 1
  tweets[target.replace('processed', '').replace('.csv', '').lower()] = list(set([x.strip() for x in  new_text.split('\n')]))
for k, v in tweets.items():
  tweets[k] = list(filter(lambda x: len(x.split()) >= 2, v))

In [5]:
tweets

{'negative': ['this is so me though',
  'haha mate its all good i love a debte i just wished the center',
  'thank you roshhh  i miss you and everyone too.  love youuu more',
  'it hurts me to see him so sad unhappy  you dont understand i have to do this. if i dont kill you, hell kil',
  'same. unhappy  i got a h stream coming up for my bday in  hours. and im still not sleeping.',
  'hi mahima! we share your concern unhappy  please dm us your contact details so that we can get in touch for a detailed.. cont',
  'i waited for you unhappy',
  'realized i havent eaten rice maghapon because i finished  packs of spicy seafood noodles + to  extra big chilimansi :(acid attack right now',
  'right i don%t see how people can just do shit like this',
  'oh shiiiiid noooooor unhappy',
  'i understand that look sad',
  'half of the time it doesnt load, the other half of the time it crashes unhappy  its embarassing when i',
  'my only one cat yang manja sangat and now someone has took him unhappy',

## Split text into tokens

In [6]:

tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
tokenized_tweets = dict()
for k, v in tweets.items():
  tokenized_tweets[k] = []
  for x in v:
    tokenized_tweets[k] += [[re.sub(r'(?<=\w)[^\s\w](?![^\s\w])', '', x) for x in tknzr.tokenize(x) if x not in string.punctuation]]


print('Example:', tokenized_tweets['positive'][:3])

Example: [['i', 'love', 'you', 'so', 'much', 'ruichan', 'smile', 'smile'], ['congratulations', 'youve', 'got', 'the', 'message', 'happy'], ['bot', 'im', 'not', 'perfect', 'happy']]


# Preprocess tweets in different ways

In [7]:
ps = PorterStemmer()
stemmed_tweets = dict()
for k, v in tokenized_tweets.items():
  stemmed_tweets[k] = []
  for sent in v:
    sentence = []
    for word in sent:
      sentence += [ps.stem(copy.copy(word))]
    stemmed_tweets[k] += [sentence]
print("Result:", stemmed_tweets['neutral'][0:3])

Result: [['mukul', 'kesavan', 'on', 'the', 'state', 'of', 'the', 'fourth', 'estat'], ['in', 'today', 'great', 'mean', 'win', 'the', 'say', 'sunanda', 'k', 'dattaray'], ['cop', 'dug', 'out', 'these', 'to', 'healer', 'who', 'were', 'buri', 'aliv', 'for', 'practis']]


In [8]:
lemmatizer = WordNetLemmatizer()
lemmed_tweets = dict()
for k, v in tokenized_tweets.items():
  lemmed_tweets[k] = []
  for sent in v:
    sentence = []
    for word in sent:
      sentence += [lemmatizer.lemmatize(copy.copy(word))]
    lemmed_tweets[k] += [sentence]
print("Result:", lemmed_tweets['neutral'][0:3])

Result: [['mukul', 'kesavan', 'on', 'the', 'state', 'of', 'the', 'fourth', 'estate'], ['in', 'today', 'greatness', 'mean', 'winning', 'the', 'say', 'sunanda', 'k', 'dattaray'], ['cop', 'dug', 'out', 'these', 'to', 'healer', 'who', 'were', 'buried', 'alive', 'for', 'practising']]


In [9]:
%%time
spelled_tokenized_tweets = dict()
spell = Speller(lang='en')
for k, v in tokenized_tweets.items():
  spelled_tokenized_tweets[k] = []
  for sent in v:
    sentence = []
    for word in sent:
      sentence += [spell(copy.copy(word))]
    spelled_tokenized_tweets[k] += [sentence]
print("Result:", spelled_tokenized_tweets['neutral'][0:3])

Result: [['mutual', 'kesavan', 'on', 'the', 'state', 'of', 'the', 'fourth', 'estate'], ['in', 'today', 'greatness', 'means', 'winning', 'the', 'says', 'uganda', 'k', 'dattaray'], ['cops', 'dug', 'out', 'these', 'to', 'headers', 'who', 'were', 'buried', 'alive', 'for', 'practising']]
CPU times: user 1min 57s, sys: 252 ms, total: 1min 57s
Wall time: 1min 57s


In [10]:
spelled_lemmed_tweets = dict()
for k, v in spelled_tokenized_tweets.items():
  spelled_lemmed_tweets[k] = []
  for sent in v:
    sentence = []
    for word in sent:
      sentence += [lemmatizer.lemmatize(copy.copy(word))]
    spelled_lemmed_tweets[k] += [sentence]
print("Result:", spelled_lemmed_tweets['neutral'][0:3])

Result: [['mutual', 'kesavan', 'on', 'the', 'state', 'of', 'the', 'fourth', 'estate'], ['in', 'today', 'greatness', 'mean', 'winning', 'the', 'say', 'uganda', 'k', 'dattaray'], ['cop', 'dug', 'out', 'these', 'to', 'header', 'who', 'were', 'buried', 'alive', 'for', 'practising']]


In [11]:
spelled_stemmed_tweets = dict()
for k, v in spelled_tokenized_tweets.items():
  spelled_stemmed_tweets[k] = []
  for sent in v:
    sentence = []
    for word in sent:
      sentence += [ps.stem(copy.copy(word))]
    spelled_stemmed_tweets[k] += [sentence]
print("Result:", spelled_stemmed_tweets['neutral'][0:3])

Result: [['mutual', 'kesavan', 'on', 'the', 'state', 'of', 'the', 'fourth', 'estat'], ['in', 'today', 'great', 'mean', 'win', 'the', 'say', 'uganda', 'k', 'dattaray'], ['cop', 'dug', 'out', 'these', 'to', 'header', 'who', 'were', 'buri', 'aliv', 'for', 'practis']]


In [12]:
sw = set(stopwords.words('english'))
my_sw = set()
for word in sw:
  my_sw.add(word.replace("\'", ''))
  my_sw.add(spell(word.replace("\'", '')))
  my_sw.add(lemmatizer.lemmatize(word.replace("\'", '')))
  my_sw.add(ps.stem(word.replace("\'", '')))
  my_sw.add(spell(lemmatizer.lemmatize(word.replace("\'", ''))))
  my_sw.add(spell(ps.stem(word.replace("\'", ''))))
sw = my_sw | sw


In [13]:
%%time
def remove_sw(sw_, data_):
  for k, v in data_.items():
    for sentence in v:
      for stop_word in sw_:
        while stop_word in sentence:
          sentence.remove(stop_word)

sw_spelled_stemmed_tweets = copy.deepcopy(spelled_stemmed_tweets)
sw_stemmed_tweets = copy.deepcopy(stemmed_tweets)
sw_lemmed_tweets = copy.deepcopy(lemmed_tweets)
sw_spelled_lemmed_tweets = copy.deepcopy(spelled_lemmed_tweets)
remove_sw(sw, sw_spelled_stemmed_tweets)
remove_sw(sw, sw_stemmed_tweets)
remove_sw(sw, sw_lemmed_tweets)
remove_sw(sw, sw_spelled_lemmed_tweets)

CPU times: user 692 ms, sys: 4.99 ms, total: 697 ms
Wall time: 697 ms


In [14]:
def vectorize_bin(s, bag_, process_method=None):
  tknzr_ = TweetTokenizer(strip_handles=True, reduce_len=True)
  vector = list()
  if isinstance(s, str):
    tokenized = [re.sub(r'(?<=\w)[^\s\w](?![^\s\w])', '', x.lower()) for x in tknzr_.tokenize(s) if x not in string.punctuation]
  elif isinstance(s, list) or isinstance(s, np.ndarray):
    tokenized = s
  if process_method:
    tokenized = [process_method(x) for x in tokenized]
  for word in bag_:
    if word in tokenized:
      vector.append(1)
    else:
      vector.append(0)
  return np.array(vector)

def vectorize_count(s, bag_, process_method=None):
  tknzr_ = TweetTokenizer(strip_handles=True, reduce_len=True)
  vector = list()
  if isinstance(s, str):
    tokens = [re.sub(r'(?<=\w)[^\s\w](?![^\s\w])', '', x.lower()) for x in tknzr_.tokenize(s) if x not in string.punctuation]
  elif isinstance(s, list) or isinstance(s, np.ndarray):
    tokens = s
  if process_method:
    tokens = [process_method(x) for x in tokens]
  vector = list()
  for word in bag_:
    count = 0
    for tok in tokens:
      if word == tok:
        count += 1
    vector.append(count)
  return np.array(vector)
    
def vectorize_tfidf(s, idf, bag_, process_method=None):
  tknzr_ = TweetTokenizer(strip_handles=True, reduce_len=True)
  vector = list()
  if isinstance(s, str):
    tokens = [re.sub(r'(?<=\w)[^\s\w](?![^\s\w])', '', x.lower()) for x in tknzr_.tokenize(s) if x not in string.punctuation]
  elif isinstance(s, list) or isinstance(s, np.ndarray):
    tokens = s
  if process_method:
    tokens = [process_method(x) for x in tokens]
  for word in bag_:
    count = 0
    for tok in tokens:
      if word == tok:
        count += 1
    tf = count / len(tokens)
    if word in idf.keys():
      vector.append(tf * idf[word])
    else:
      vector.appen(tf * max(idf.values()))
  return np.array(vector)

def split(data, vectorize=vectorize_bin, spt=True, process_method=None):
  xtrain = copy.deepcopy(data['positive'][:int(len(data['positive']) * 0.8)]) + \
            copy.deepcopy(data['neutral'][:int(len(data['neutral']) * 0.8)]) + \
            copy.deepcopy(data['negative'][:int(len(data['negative']) * 0.8)])

  ytrain = [0] * len(data['positive'][:int(len(data['positive']) * 0.8)]) + \
            [1] * len(data['neutral'][:int(len(data['neutral']) * 0.8)]) + \
            [2] * len(data['negative'][:int(len(data['negative']) * 0.8)])

  xtest = copy.deepcopy(data['positive'][int(len(data['positive']) * 0.8):]) + \
          copy.deepcopy(data['neutral'][int(len(data['neutral']) * 0.8):]) + \
          copy.deepcopy(data['negative'][int(len(data['negative']) * 0.8):])
          
  ytest = [0] * len(data['positive'][int(len(data['positive']) * 0.8):]) + \
          [1] * len(data['neutral'][int(len(data['neutral']) * 0.8):]) + \
          [2] * len(data['negative'][int(len(data['negative']) * 0.8):])

  bag = set()
  for x in xtrain if spt else xtrain + xtest:
    for y in x:
      bag.add(y)
  bag = list(bag)

  if vectorize == vectorize_tfidf:
    idf = collections.Counter()
    n = 0
    for k, v in data.items():
      n += len(v)
    for _, sentences in data.items():
      for word in bag:
        for sentence in sentences:
          if word in sentence:
            idf[word] += 1
    for k, v in idf.items():
      idf[k] = math.log(n / idf[k])

  i = 0
  while i < len(xtrain):
    xtrain[i] = vectorize(xtrain[i], bag, process_method) if vectorize is not vectorize_tfidf else vectorize(xtrain[i], idf, bag, process_method)
    i += 1
  i = 0
  while i < len(xtest):
    xtest[i] = vectorize(xtest[i], bag, process_method) if vectorize is not vectorize_tfidf else vectorize(xtest[i], idf, bag, process_method)
    i += 1
  if spt and vectorize is not vectorize_tfidf:
    return np.array(xtrain), np.array(ytrain), np.array(xtest), np.array(ytest), np.array(bag)
  elif vectorize is not vectorize_tfidf:
    return xtrain + xtest, ytrain + ytest, bag
  elif spt and vectorize is vectorize_tfidf:
    return np.array(xtrain), np.array(ytrain), np.array(xtest), np.array(ytest), np.array(bag), idf
  else:
    return xtrain + xtest, ytrain + ytest, bag, idf

In [15]:
%%time
if GRID_SEARCH:
  params = {'max_depth': [4, 10, 30, 50, 70, 130, 143, 150, 170, 177],
                'n_estimators': (5, 10, 43, 50, 73, 200),
                'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None]}
  search = GridSearchCV(RandomForestClassifier(random_state=42), params,  scoring='accuracy', n_jobs=-1)
  x, y, bag = split(tokenized_tweets, vectorize_bin, False)
  search.fit(x, y)
  print(search.best_params_)

def class_(number):
  return 'negative' if number == 2 else 'neutral' if number == 1 else 'positive'

# {'class_weight': None, 'criterion': 'entropy', 'max_depth': 150, 'n_estimators': 50}
# {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 170, 'n_estimators': 200}
# CPU times: user 14min 18s, sys: 3min 7s, total: 17min 26s
# Wall time: 56min 33s

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11 µs


In [16]:
def crossval(n_splits, X, y, model):
    train_acc = []
    test_acc = []
    X = np.array(X)
    y = np.array(y)
    kf = KFold(n_splits=n_splits, random_state=21, shuffle=True)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        y_predict_train = model.predict(X_train)
        y_predict_test = model.predict(X_test)
        test = accuracy_score(y_predict_test, y_test)
        train  = accuracy_score(y_predict_train, y_train)
        print("train - ", round(train, 5), '\t|\t', "valid - ", round(test, 5))
        train_acc += [train]
        test_acc += [test]
    print("Average accuracy on crossval is", round(np.array(test_acc).mean(), 5))
    print("Std is ", round(np.array(test_acc).std(), 5))
    return np.array(test_acc).mean(), np.array(test_acc).std()

def choose_best_preprocessing(methods):
  best_model = ['', vectorize_bin, 0]
  for name, data in methods.items():
    x, y, bag_ = split(copy.deepcopy(data), vectorize=vectorize_bin, spt=False)
    model = RandomForestClassifier(random_state=42, 
                                  class_weight='balanced',
                                  criterion='gini',
                                  max_depth=170,
                                  n_estimators=200,
                                  n_jobs=-1)
    acc, std = crossval(5, x, y, model)
    if acc > best_model[2]:
      best_model[0] = name
      best_model[1] = vectorize_bin
      best_model[2] = acc
    print(f'({name}, binarized vector) accuracy:', acc)
    x, y, bag_ = split(copy.deepcopy(data), vectorize=vectorize_count, spt=False)
    model = RandomForestClassifier(random_state=42, 
                                  class_weight='balanced',
                                  criterion='gini',
                                  max_depth=170,
                                  n_estimators=200,
                                  n_jobs=-1)
    acc, std = crossval(5, x, y, model)
    if acc > best_model[2]:
      best_model[0] = name
      best_model[1] = vectorize_count
      best_model[2] = acc
    print(f'({name}, counted words) accuracy:', acc)
    x, y, bag_, idf = split(copy.deepcopy(data), vectorize=vectorize_tfidf, spt=False)
    model = RandomForestClassifier(random_state=42, 
                                  class_weight='balanced',
                                  criterion='gini',
                                  max_depth=170,
                                  n_estimators=200,
                                  n_jobs=-1)
    acc, std = crossval(5, x, y, model)
    if acc > best_model[2]:
      best_model[0] = name
      best_model[1] = vectorize_tfidf
      best_model[2] = acc
    print(f'({name}, tfidf vector) accuracy:', acc)
    print("ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡")
  return best_model

#Using cross validation select best preprocessing method

In [17]:
%%time
if SELECT_BEST_PREPOCESS:
  methods = {"Just tokenized": copy.deepcopy(tokenized_tweets),
            "Stemmed": copy.deepcopy(stemmed_tweets),
            "Lemmatized": copy.deepcopy(lemmed_tweets),
            "Spelled and tokenized": copy.deepcopy(spelled_tokenized_tweets),
            "Spelled and lemmatized": copy.deepcopy(spelled_lemmed_tweets),
            "Spelled and stemmed": copy.deepcopy(spelled_stemmed_tweets),
            "Spelled and stemmed (stopword removes)": copy.deepcopy(sw_spelled_stemmed_tweets),
            "Stemmed (stopwords removing": copy.deepcopy(sw_stemmed_tweets),
            "Lemmatized (stopwords removed)": copy.deepcopy(sw_lemmed_tweets),
            "Spelled and lemmatized (stopword removed)": copy.deepcopy(sw_spelled_lemmed_tweets)}
  best_model = choose_best_preprocessing(methods)
  print(best_model[0], end=' ')
  if best_model[1] == vectorize_bin:
    print("vectorize_bin")
  elif best_model[1] == vectorize_count:
    print("vectorize_count")
  else:
    print("vectorize_tfidf")
  print("best accuracy:", best_model[2])
if GRID_SEARCH_BEST:
  print("Starting grid search")
  params = {'max_depth': [61, 130, 143, 150, 170, 177, 189, 201],
                'n_estimators': (43, 50, 73, 200, 231, 253),
                'criterion': ('entropy', 'gini'),
                'class_weight': ['balanced', None]}
  search = GridSearchCV(RandomForestClassifier(random_state=42), params,  scoring='accuracy', n_jobs=-1, verbose=2)
  if best_model[1] is not vectorize_tfidf:
    x, y, bag = split(methods[best_model[0]], best_model[1], False)
  else:
    x, y, bag, idf = split(methods[best_model[0]], best_model[1], False)
  search.fit(x, y)
  print(search.best_params_)


train -  1.0 	|	 valid -  0.93573
train -  1.0 	|	 valid -  0.91871
train -  1.0 	|	 valid -  0.94518
train -  1.0 	|	 valid -  0.95085
train -  1.0 	|	 valid -  0.94707
Average accuracy on crossval is 0.93951
Std is  0.01153
(Just tokenized, binarized vector) accuracy: 0.9395085066162572
train -  1.0 	|	 valid -  0.93195
train -  1.0 	|	 valid -  0.91304
train -  1.0 	|	 valid -  0.94518
train -  1.0 	|	 valid -  0.94707
train -  1.0 	|	 valid -  0.95274
Average accuracy on crossval is 0.938
Std is  0.01422
(Just tokenized, counted words) accuracy: 0.9379962192816634
train -  1.0 	|	 valid -  0.93195
train -  1.0 	|	 valid -  0.91871
train -  1.0 	|	 valid -  0.93384
train -  1.0 	|	 valid -  0.93573
train -  1.0 	|	 valid -  0.94707
Average accuracy on crossval is 0.93346
Std is  0.00906
(Just tokenized, tfidf vector) accuracy: 0.9334593572778829
ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡
train -  1.0 	|	 valid -  0.93573
train -  1.0 	|	 valid 

In [18]:
# {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 150, 'n_estimators': 253, 'n_jobs': -1, 'random_state': 42}
# {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 143, 'n_estimators': 200, 'n_jobs': -1, 'random_state': 42}
# old versions searched on binary vectors:
# {'class_weight': None, 'criterion': 'entropy', 'max_depth': 150, 'n_estimators': 50, 'n_jobs': -1, 'random_state': 42}
# {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 170, 'n_estimators': 200, 'n_jobs': -1, 'random_state': 42}
if 'search' in locals().keys():
  print('Getting best params from search ...')
  best_params = copy.deepcopy(search.best_params_)
  best_params['n_jobs'] = -1
  best_params['random_state'] = 42
  best_preprocessed_dataset = copy.deepcopy(methods[best_model[0]])
  vectorizatin_method = best_model[1]
else:
  best_params = {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 150, 'n_estimators': 253, 'n_jobs': -1, 'random_state': 42}
  if 'best_model' in locals().keys():
    vectorizatin_method = best_model[1]
    best_preprocessed_dataset = copy.deepcopy(methods[best_model[0]])
  else:
    vectorizatin_method = vectorize_bin
    best_preprocessed_dataset = copy.deepcopy(spelled_lemmed_tweets)
  
model = RandomForestClassifier(**best_params)
if best_model[1] is not vectorize_tfidf:
  xtrain, ytrain, xtest, ytest, bag_ = split(best_preprocessed_dataset, vectorize=vectorizatin_method, spt=True)
else:
  xtrain, ytrain, xtest, ytest, bag_, idf = split(best_preprocessed_dataset, vectorize=vectorizatin_method, spt=True)
model.fit(xtrain, ytrain)
print("final accuracy:", accuracy_score(model.predict(xtest), ytest))

final accuracy: 0.9415094339622642


#Building Word2Vec model

In [19]:
%%time
embeddings = gensim.models.Word2Vec(size=300, min_count=1, workers=4)
sentences = reduce(lambda x, y: x + y, tokenized_tweets.values())
embeddings.build_vocab(sentences)
embeddings.train(sentences=sentences, total_examples=len(sentences), epochs=4000)

CPU times: user 6min 19s, sys: 5.41 s, total: 6min 25s
Wall time: 3min 45s


After that we can calculate simalarity of 2 words or find most similar

In [20]:
print('memoriam', embeddings.wv.most_similar('memoriam', topn=3))
print('college', embeddings.wv.most_similar('college', topn=10))
print('twitter', embeddings.wv.most_similar('twitter', topn=3))
print(embeddings.wv.similarity('i', 'you'))
print('i', embeddings.wv.most_similar('i', topn=3))

memoriam [('alea', 0.938609778881073), ('starbridge', 0.8638456463813782), ('phaseiii', 0.6975321173667908)]
college [('general', 0.43792229890823364), ('nervous', 0.4279614984989166), ('fund', 0.4263381063938141), ('protests', 0.4248746633529663), ('ffsrt', 0.41493359208106995), ('poaching', 0.40895888209342957), ('zplus', 0.40712201595306396), ('outfit', 0.40454643964767456), ('initiatives', 0.4021744132041931), ('fetch', 0.4006943106651306)]
twitter [('daianeryoufato', 0.363331139087677), ('administrator', 0.34377598762512207), ('uncheck', 0.33018723130226135)]
0.6628305
i [('unhappy', 0.6656458377838135), ('you', 0.6628304719924927), ('to', 0.648640513420105)]


In [21]:
def train_test_split(data):
  xtr = copy.deepcopy(data['positive'][:int(len(data['positive']) * 0.8)]) + \
            copy.deepcopy(data['neutral'][:int(len(data['neutral']) * 0.8)]) + \
            copy.deepcopy(data['negative'][:int(len(data['negative']) * 0.8)])

  ytr = [0] * len(data['positive'][:int(len(data['positive']) * 0.8)]) + \
            [1] * len(data['neutral'][:int(len(data['neutral']) * 0.8)]) + \
            [2] * len(data['negative'][:int(len(data['negative']) * 0.8)])

  xte = copy.deepcopy(data['positive'][int(len(data['positive']) * 0.8):]) + \
          copy.deepcopy(data['neutral'][int(len(data['neutral']) * 0.8):]) + \
          copy.deepcopy(data['negative'][int(len(data['negative']) * 0.8):])
          
  yte = [0] * len(data['positive'][int(len(data['positive']) * 0.8):]) + \
          [1] * len(data['neutral'][int(len(data['neutral']) * 0.8):]) + \
          [2] * len(data['negative'][int(len(data['negative']) * 0.8):])
  return xtr, ytr, xte, yte

def w2c_sentence(sent, embeddings):
  word_vectors = []
  for word in sent:
    word_vectors += [embeddings.wv[word]]
  return np.array(word_vectors).mean(axis=0)


In [22]:
w2c = dict()
for sentiment, sentences in tokenized_tweets.items():
  w2c[sentiment] = []
  for sentence in sentences:
    w2c[sentiment] += [w2c_sentence(sentence, embeddings)]

In [23]:
xtr, ytr, xte, yte = train_test_split(w2c)
model = RandomForestClassifier(**best_params)
model.fit(xtr, ytr)
print("accuracy:", accuracy_score(model.predict(xte), yte))

accuracy: 0.8490566037735849


In [24]:
model = CatBoostClassifier()
model.fit(xtr, ytr, silent=True)
print()
print("accuracy:", accuracy_score(model.predict(xte), yte))


accuracy: 0.8716981132075472


Find similar tweets with cosine similarity metric


In [25]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    return collections.Counter(text)


def exist(sentences, similar, sim):
  for x in similar:
    if sentences[x[1]] == sentences[sim[1]] or sentences[x[2]] == sentences[sim[1]] or sentences[x[1]] == sentences[sim[2]] or sentences[x[2]] == sentences[sim[2]]:
      return True
  return False


def top_similar(sentences, n_sim=10):
  sentences = list(map(text_to_vector, sentences))
  similar = []
  i = 0
  j = 0
  with tqdm(total=((len(sentences))* (len(sentences) - 1) / 2)) as tq:
    while i < len(sentences):
      j = i + 1
      while j < len(sentences):
        if j != i:
          sim = [get_cosine(sentences[i], sentences[j]), i, j]
          if similar and len(similar) >= n_sim:
            if similar[-1][0] < sim[0] and sim[0] <= .95 and not exist(sentences, similar, sim):
                similar[-1] = sim
                similar = sorted(similar, key=lambda x: x[0], reverse=True)
          elif sim[0] <= .95 and not exist(sentences, similar, sim):
              similar += [sim]
              similar = sorted(similar, key=lambda x: x[0], reverse=True)
        j += 1
        tq.update(1)
      i += 1

  for x in similar:
    print(x[0])
    print(' '.join(sentences[x[1]]))
    print(' '.join(sentences[x[2]]))
    print()

In [26]:
methods = {"Just tokenized": copy.deepcopy(tokenized_tweets),
          "Stemmed": copy.deepcopy(stemmed_tweets),
          "Lemmatized": copy.deepcopy(lemmed_tweets),
          "Spelled and tokenized": copy.deepcopy(spelled_tokenized_tweets),
          "Spelled and lemmatized": copy.deepcopy(spelled_lemmed_tweets),
          "Spelled and stemmed": copy.deepcopy(spelled_stemmed_tweets),
          "Spelled and stemmed (stopword removes)": copy.deepcopy(sw_spelled_stemmed_tweets),
          "Stemmed (stopwords removing": copy.deepcopy(sw_stemmed_tweets),
          "Lemmatized (stopwords removed)": copy.deepcopy(sw_lemmed_tweets),
          "Spelled and lemmatized (stopword removed)": copy.deepcopy(sw_spelled_lemmed_tweets)}

for name, data in methods.items():
  print(name)
  top_similar(reduce(lambda x, y: x + y, copy.deepcopy(data).values()))
  print("ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡ ¯\_(ツ)_/¯ (─‿‿─) 〈( ^.^)ノ ʕ•́ᴥ•̀ʔっ♡")
  print()

Just tokenized


100%|██████████| 3496690/3496690.0 [00:48<00:00, 72702.52it/s]


0.9486832980505138
for the recent follow much appreciated happy want this
thanks for the recent follow much appreciated happy want this

0.9486832980505138
what got when it bred local pig with variety
what meghalaya got when it bred local pig with variety

0.9411764705882353
thanks for the recent follow happy to connect have a great thursday want it
thanks for the recent follow happy to connect have a great thursday want this

0.9393364366277241
thanks for the recent follow happy to connect have a great wednesday want this
thanks for the recent follow happy to connect have a great wednesday

0.9136643851522572
hi we tried to call your number but got no response unhappy please share another suitable time and an alternate for us .. cont
hi ashish we tried to call your number but got no response unhappy please share another suitable time and an alternate .. cont

0.9128709291752769
share the love high value members of this week happy insight by
share the love high value members of this we

100%|██████████| 3496690/3496690.0 [00:48<00:00, 72173.77it/s]


0.9486832980505138
what got when it bred local pig with varieti
what meghalaya got when it bred local pig with varieti

0.9393364366277241
thank for the recent follow happi to connect have a great wednesday want thi
thank for the recent follow happi to connect have a great wednesday

0.9136643851522572
hi we tri to call your number but got no respons unhappi pleas share anoth suitabl time and an altern for us .. cont
hi ashish we tri to call your number but got no respons unhappi pleas share anoth suitabl time and an altern .. cont

0.9128709291752769
share the love high valu member of thi week happi insight by
share the love high valu member of thi week happi

0.9128709291752769
i miss so much unhappi
i miss him so much unhappi

0.9128709291752769
feel so fuck shit today unhappi
feel so shit today unhappi

0.8970852271450606
thank for be top engag commun member thi week happi want
thank for be top engag commun member thi week happi tri too

0.8944271909999159
thank for the recent foll

...

0.9428090415820632
thank top engag commun member week happi want
thank top engag commun member week happi want free

0.9354143466934852
share love high valu member week happi insight
share love high valu member week happi

0.9309493362512627
talk driver im goingh said hed love go new york sinc trump probabl
talk driver im goingh said hed love go new york sinc trump probabl pleas unhappi

0.9258200997725514
thank recent follow much appreci happi want
thank recent follow much appreci happi

0.9128709291752769
got bred local pig varieti
meghalaya got bred local pig varieti

0.9128709291752769
tri call number got respons unhappi pleas share anoth suitabl time altern us .. cont
ashish tri call number got respons unhappi pleas share anoth suitabl time altern .. cont

0.8944271909999159
happi everyon
happi

0.8944271909999159
new amazingphil video today happi
new video today happi

0.8944271909999159
die thirst us unhappi
koala die thirst us unhappi

0.8944271909999159
feel fuck shit today un

100%|██████████| 3496690/3496690.0 [00:35<00:00, 97242.37it/s] 


0.9428090415820632
thanks top engaged community member week happy want
thanks top engaged community member week happy want free

0.9354143466934852
thanks retweets week much appreciated happy want
thanks retweets week much appreciated happy want free

0.9354143466934852
share love high value member week happy insight
share love high value member week happy

0.9309493362512627
talking driver im goinghe said hed love go new york since trump probably
talking driver im goinghe said hed love go new york since trump probably please unhappy

0.9258200997725514
thanks recent follow much appreciated happy want
thanks recent follow much appreciated happy

0.9128709291752769
thanks recent follow happy connect great thursday want free
thanks recent follow happy connect great thursday

0.9128709291752769
got bred local pig variety
meghalaya got bred local pig variety

0.9128709291752769
tried call number got response unhappy please share another suitable time alternate u .. cont
ashish tried call n

100%|██████████| 3496690/3496690.0 [00:35<00:00, 97383.59it/s]

0.9428090415820632
thanks top engaged community member week happy want
thanks top engaged community member week happy want free

0.9354143466934852
share love high value member week happy insight
share love high value member week happy

0.9309493362512627
talking driver im going said hed love go new york since trump probably
talking driver im going said hed love go new york since trump probably please unhappy

0.9258200997725514
thanks recent follow much appreciated happy want
thanks recent follow much appreciated happy

0.9128709291752769
thanks recent follow happy connect great thursday want free
thanks recent follow happy connect great thursday

0.9128709291752769
got bred local pig variety
meghalaya got bred local pig variety

0.9128709291752769
tried call number got response unhappy please share another suitable time alternate u .. cont
assist tried call number got response unhappy please share another suitable time alternate .. cont

0.9090909090909091
thanks recent follow happy 